In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
sns.set(style='whitegrid')

In [ ]:
import os
from os.path import join

# Introduction

## Aim

Compute the results of the analysis of the 0D model sensitivity analysis. In particular, implement the pipeline to extract sensitivity analysis data, save latex tables. 

## Prerequisites

* Run the script `script_sensitivity_analysis.py` for several values of epsilon
    * Run in folders with name format: `epsilon_<value>` (e.g. `epsilon_0.01`)
* For each generated subfolders in `epsilon_<value>/`, process with the script `test_dnnmodelevaluation.m` and copy the `table.csv`, `Xexact.csv` and `Xpredicted.csv` files
* Each `epsilon_<value>/` folder should contain the following (contents of `outputs/` directory is not shown): 

```
.
├── P0_Param_LeftVentricle_Emax0
│   ├── outputs
│   ├── parameters.txt
│   ├── table.csv
│   ├── Xexact.csv
│   └── Xpredicted.csv
├── P1_Param_LeftVentricle_EmaxRef0
│   ├── outputs
│   ├── parameters.txt
│   ├── table.csv
│   ├── Xexact.csv
│   └── Xpredicted.csv
├── P2_Param_LeftVentricle_AGain_Emax
│   ├── outputs
│   ├── parameters.txt
│   ├── table.csv
│   ├── Xexact.csv
│   └── Xpredicted.csv
└── P3_Param_LeftVentricle_kE
    ├── outputs
    ├── parameters.txt
    ├── table.csv
    ├── Xexact.csv
    └── Xpredicted.csv
```

## How to use

**For each value of $\mathbf\epsilon$, you should rerun the notebook and modify the `identifier` accordingly** (see the cell below).

## Setup



In [ ]:
identifier = 'epsilon_0.2'
path = f'/media/maousi/Data/tmp/sensitivity/{identifier}/'

# Check data

## Aim

Verify that simulations in each directory actually has a single varying parameter. Visualize parameters distribution.

In [ ]:
# Data containing values of the parameter that varies in a given folder
param_data = {}

for folder in os.listdir(path):
    print(f'=== {folder}')
    file = join(path, folder, 'parameters.txt')
    # Check file contents (that indeed a single parameter varies)
    #!head -4 $file | tail -3
    
    param_name = folder[folder.find('_')+1:]
    df = pd.read_csv(file)
    print(df.head(4).values)
    param_data[param_name] = df[param_name].values

In [ ]:
fig, axes = plt.subplots(1, len(param_data), figsize=(16,5))

paramslatex = dict(zip(param_data.keys(), ['$E_{max,lv}$', '$E_{max,lv,0}$', '$G_{E_{max,lv}}$', '$k_{E,lv}$']))
for p, ax in zip(param_data, axes):
    sns.distplot(param_data[p], ax=ax, norm_hist=True)
    ax.set_title(paramslatex[p])

# Extract values

In [ ]:
# Define utility function to format results table
def format_table(table):
    table.index = [
        'Min exact', 'Max exact', 'Mean exact', 'SD exact', 'Min pred', 
        'Max pred', 'Mean pred', 'SD pred', 'Avg err mx', 
        'Avg rel err%', 'SD error', 'CI min', 'CI max',
    ]
    table.columns = [
        'HR','SAPM','SAPS','SAPD', 'PAPM','PAPS','PAPD','LVEF',
        'LVEDV', 'LVESV', 'CI', 'PCPW'
    ]
    table.CI = table.CI.apply(np.abs)
    cols = {'Min exact': 'Min', 'Max exact': 'Max', 'Mean exact': 'Mean', 'SD exact': 'SD'}
    table = table.loc[list(cols.keys()), :]
    table = table.rename(cols)
    #table.index = table.index.set_levels(['Emax0','EmaxRef0','GainEmax','kE'], level=0)
    #table.index = ['Emax0','EmaxRef0','GainEmax','kE']
    #table.index.rename({'Param_LeftVentricle_Emax0':'test'}, inplace=True, level=0)
    return table

In [ ]:
raw = {}
for folder in os.listdir(path):
    file = join(path, folder, 'table.csv')
    print(file)
    df = pd.read_csv(file, header=None)
    df = format_table(df)
    param_name = folder[folder.find('_')+1:]
    raw[param_name] = df

In [ ]:
df = pd.concat(raw)
df.index = df.index.set_levels(['$E_{max,lv}$', '$E_{max,lv,0}$', 
                                '$G_{E_{max,lv}}$', '$k_{E,lv}$'], level=0)
df

In [ ]:
df.to_latex(f'data/sensitivity_analysis_{identifier}.tex', float_format="%.4g", 
            multirow=True, escape=False, caption=identifier)#, longtable=True)

# Plot parameter distribution



In [ ]:
path = '/media/maousi/Data/tmp/sensitivity'
params = []
# For each value of epsilon
for folder in os.listdir(path):
    print(folder)
    # Extract epsilon value
    epsilon = folder.split('_')[-1]
    epsilon = float(epsilon)
    print(epsilon)
    # For each 0D model parameter
    dfepsilon = []
    for subfolder in os.listdir(join(path, folder)):
        print(subfolder)
        # Extract parameter name
        param = subfolder[3:]
        print(param)
        # Read parameters.txt file
        pathtmp = join(path, folder, subfolder)
        print(os.listdir(join(path, folder, subfolder)))
        dftmp = pd.read_csv(join(pathtmp, 'parameters.txt'))
        print(param in dftmp.columns)
        # Extract the column of the varying parameter
        dfepsilon.append(dftmp[param])
    
    # Merge the parameter values for a given epsilon
    dfepsilon = pd.DataFrame(dfepsilon).transpose()
    # Spefiy the value of epsilon
    dfepsilon['epsilon'] = epsilon
    params.append(dfepsilon)

# Merge everything
params = pd.concat(params, ignore_index=True)

In [ ]:
names = [p for p in params.columns if p != 'epsilon']
names = list(zip(names, ['$E_{max,lv}$', '$E_{max,lv,0}$', '$G_{E_{max,lv}}$', '$k_{E,lv}$']))
names

In [ ]:
epsilons = params.epsilon.unique()
epsilons

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.ravel()

for (p, platex), ax in zip(names, axes):
    for epsilon in epsilons:
        sns.distplot(params.loc[params.epsilon == epsilon, p], 
                     ax=ax, hist=False, 
                     label=r'$\epsilon = ' + str(epsilon) + '$')
    ax.set_xlabel(platex)
        
plt.subplots_adjust(hspace=.35, wspace=.3)